# 🌞 Example Weather App
> Using `prodb`


* Starting with a `.csv` of locations, we wish to call the [MetaWeather](https://www.metaweather.com/api/) API to pull in the weather forecast.  
* The app needs the ability to input new locations by the user

 🌍 Core functions will be converted into the streamlit app `weather_app.py`

In [ ]:
#hide
from nbdev.showdoc import *
import pandas as pd
%load_ext autoreload
%autoreload 2

## 1. Requests to `metaweather` open weather API

In [ ]:
import requests
import json
import arrow

In [ ]:
def get_json_from_query(location):
    """Search for a city and return metadata from API"""
    url = f"https://www.metaweather.com/api/location/search/?query={location}"
    r = requests.get(url).json()
    return r[0]

l = get_json_from_query('London')
l

{'title': 'London',
 'location_type': 'City',
 'woeid': 44418,
 'latt_long': '51.506321,-0.12714'}

Check `woeid`(Where On Earth ID) is correct for London

In [ ]:
assert l['woeid'] == 44418

Get weather data for the London `woeid`

In [ ]:
utc = arrow.utcnow().format('YYYY/MM/DD')
utc

'2021/11/02'

In [ ]:
def get_current_weather(location):
    """
    inputs: location str "London"
            to find woeid i.e. 44418
    """
    res = get_json_from_query(location)
    woeid = res['woeid']
    url = f"https://www.metaweather.com/api/location/{woeid}/"
    res = requests.get(url).json()
    return res

Today's forecast 🌤️

In [ ]:
%%time
res = get_current_weather('London')
res['consolidated_weather'][0]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 574 ms


{'id': 6040109691437056,
 'weather_state_name': 'Light Cloud',
 'weather_state_abbr': 'lc',
 'wind_direction_compass': 'WSW',
 'created': '2021-11-02T18:59:03.647580Z',
 'applicable_date': '2021-11-02',
 'min_temp': 5.52,
 'max_temp': 11.844999999999999,
 'the_temp': 11.219999999999999,
 'wind_speed': 2.820238635213023,
 'wind_direction': 257.5126378634312,
 'air_pressure': 999.0,
 'humidity': 71,
 'visibility': 8.2086241350513,
 'predictability': 70}

There are a number of metrological properties available to us from the API. 

The ones we are most interested in are:
* `the_temp` 🌡️ current temperature
* `weather_state_name` ⛅ current sky condition
* `max_temp` 🥵 daily max temp
* `min_temp` 🥶 daily min temp


In [ ]:
res = get_current_weather('London')
df =  pd.DataFrame.from_records(res['consolidated_weather'][:1])
df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
df

,id,weather_state_name,weather_state_abbr,wind_direction_compass,created,applicable_date,min_temp,max_temp,the_temp,wind_speed,wind_direction,air_pressure,humidity,visibility,predictability,readable_time
0,6040109691437056,Light Cloud,lc,WSW,2021-11-02T18:59:03.647580Z,2021-11-02,5.52,11.845,11.22,2.820239,257.512638,999.0,71,8.208624,70,31 minutes ago


In [ ]:
def df_from_loc(location, 
                days_ahead=1,
                keep_cols='location the_temp readable_time created applicable_date local_time latt_long weather_state_name min_temp max_temp'.split(' ')):
    res = get_current_weather(location)
    df =  pd.DataFrame.from_records(res['consolidated_weather'][:days_ahead])
    df['location'] = location
    df['local_time'] = res['time']
    df['latt_long'] = res['latt_long']
    df['readable_time'] = df.created.apply(lambda x: arrow.get(x).humanize())
    return df[keep_cols]

df = df_from_loc('London', days_ahead=2)
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,11.22,32 minutes ago,2021-11-02T18:59:03.647580Z,2021-11-02,2021-11-02T19:31:17.474297Z,"51.506321,-0.12714",Light Cloud,5.52,11.845
1,London,9.82,32 minutes ago,2021-11-02T18:59:02.463365Z,2021-11-03,2021-11-02T19:31:17.474297Z,"51.506321,-0.12714",Showers,4.40,9.745


Example output showing location and current weather situation

In [ ]:
df = df_from_loc('London')
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,11.22,32 minutes ago,2021-11-02T18:59:03.647580Z,2021-11-02,2021-11-02T19:31:20.442624Z,"51.506321,-0.12714",Light Cloud,5.52,11.845


In [ ]:
df = df_from_loc('Auckland')
df

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,18.19,2 hours ago,2021-11-02T16:37:04.568230Z,2021-11-03,2021-11-03T08:31:22.154620+13:00,"-36.884109,174.770416",Showers,10.92,17.31


# 🦄 Pro db 

In [ ]:
import sys
sys.path.append('../')

from prodb.core import generate_db, insert_row

### Initialise empty db

In [ ]:
dbpath = 'db.csv'
cols = 'location temp high low'.split()

generate_db(cols=cols, dbpath=dbpath)
df = pd.read_csv('db.csv')
df.head()

✓💾 db.csv (23 kb)


,location,temp,high,low


## Add row

In [ ]:
data = {'location':'London', 
        'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
        'temp': '11',
        'high':'13',
        'low':'6'}
df = insert_row(df, data)
display(df)

,location,temp,high,low,time_utc
0,London,11,13,6,2021-11-02 22:16:26


In [ ]:
dx = df_from_loc('Auckland')
dx

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Auckland,18.19,2 hours ago,2021-11-02T19:37:05.357619Z,2021-11-03,2021-11-03T11:16:27.668899+13:00,"-36.884109,174.770416",Showers,10.75,17.215


In [ ]:
dx = df_from_loc('Christchurch')

data = {'location': dx.location.item(),
        'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
        'temp': dx.the_temp.item(),
        'high': dx.min_temp.item(),
        'low': dx.max_temp.item()}

df = insert_row(df, data)
display(df)

,location,temp,high,low,time_utc
0,London,11,13,6,2021-11-02 22:16:26
1,Christchurch,12.53,5.96,12.865,2021-11-02 22:16:29


In [ ]:

def visit_city(df, cities):
    for city in cities:
        dx = df_from_loc(city).round(1)

        data = {'location': dx.location.item(), 
                'time_utc': arrow.utcnow().format('YYYY-MM-DD HH:mm:ss'),
                'temp': dx.the_temp.item(),
                'high': dx.min_temp.item(),
                'low': dx.max_temp.item()}

        df = insert_row(df, data)
    return df
        
df = visit_city(df, ['Wellington', 'Lagos', 'Zagreb'])
display(df)

,location,temp,high,low,time_utc
0,London,11,13,6,2021-11-02 22:16:26
1,Christchurch,12.53,5.96,12.865,2021-11-02 22:16:29
2,Wellington,12.8,10.2,13.5,2021-11-02 22:16:30
3,Lagos,29.4,25.0,30.9,2021-11-02 22:16:30
4,Zagreb,11.2,7.0,12.2,2021-11-02 22:16:31


In [ ]:
df_from_loc('London').round(1)

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,London,11.0,17 minutes ago,2021-11-02T21:59:01.656858Z,2021-11-02,2021-11-02T22:16:32.578764Z,"51.506321,-0.12714",Light Cloud,4.9,11.8


In [ ]:
df = visit_city(df, ['Bangkok'])
df_from_loc('Bangkok').round(1)

,location,the_temp,readable_time,created,applicable_date,local_time,latt_long,weather_state_name,min_temp,max_temp
0,Bangkok,31.7,2 hours ago,2021-11-02T19:39:20.557470Z,2021-11-03,2021-11-03T05:16:33.813841+07:00,"13.753330,100.504822",Heavy Rain,24.0,31.8


In [ ]:
df.time_utc.apply(lambda x: arrow.get(x).humanize())

0    just now
1    just now
2    just now
3    just now
4    just now
5    just now
Name: time_utc, dtype: object

In [ ]:
df['human'] = df.time_utc.apply(lambda x: arrow.get(x).humanize())
display(df)

,location,temp,high,low,time_utc,human
0,London,11,13,6,2021-11-02 22:16:26,30 seconds ago
1,Christchurch,12.53,5.96,12.865,2021-11-02 22:16:29,27 seconds ago
2,Wellington,12.8,10.2,13.5,2021-11-02 22:16:30,26 seconds ago
3,Lagos,29.4,25.0,30.9,2021-11-02 22:16:30,26 seconds ago
4,Zagreb,11.2,7.0,12.2,2021-11-02 22:16:31,25 seconds ago
5,Bangkok,31.7,24.0,31.8,2021-11-02 22:16:33,23 seconds ago


In [ ]:
df = visit_city(df, ['Manila'])
df['human'] = df.time_utc.apply(lambda x: arrow.get(x).humanize())
display(df)

,location,temp,high,low,time_utc,human
0,London,11,13,6,2021-11-02 22:16:26,a minute ago
1,Christchurch,12.53,5.96,12.865,2021-11-02 22:16:29,57 seconds ago
2,Wellington,12.8,10.2,13.5,2021-11-02 22:16:30,56 seconds ago
3,Lagos,29.4,25.0,30.9,2021-11-02 22:16:30,56 seconds ago
4,Zagreb,11.2,7.0,12.2,2021-11-02 22:16:31,55 seconds ago
5,Bangkok,31.7,24.0,31.8,2021-11-02 22:16:33,53 seconds ago
6,Manila,32.1,24.8,32.0,2021-11-02 22:17:26,just now


In [ ]:
df = visit_city(df, ['Alexandria'])
df['human'] = df.time_utc.apply(lambda x: arrow.get(x).humanize())
display(df)

,location,temp,high,low,time_utc,human
0,London,11,13,6,2021-11-02 22:16:26,8 minutes ago
1,Christchurch,12.53,5.96,12.865,2021-11-02 22:16:29,8 minutes ago
2,Wellington,12.8,10.2,13.5,2021-11-02 22:16:30,8 minutes ago
3,Lagos,29.4,25.0,30.9,2021-11-02 22:16:30,8 minutes ago
4,Zagreb,11.2,7.0,12.2,2021-11-02 22:16:31,8 minutes ago
5,Bangkok,31.7,24.0,31.8,2021-11-02 22:16:33,8 minutes ago
6,Manila,32.1,24.8,32.0,2021-11-02 22:17:26,7 minutes ago
7,Lima,19.2,15.7,20.9,2021-11-02 22:20:45,4 minutes ago
8,alexandria,26.0,19.7,25.8,2021-11-02 22:25:25,just now
